# Meta analyses

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import functions as fun
from nistats import thresholding
import numpy as np
import nibabel as nib
import nilearn
from nilearn import masking, plotting
import matplotlib.pyplot as plt
from nimare.dataset import Dataset


template = nilearn.datasets.load_mni152_template()
affine = template.affine

level = .05
height_control = 'fdr'
cluster_threshold = 1
sample_size=119

cut_coords=(0, 0, 0)

191029-02:28:38,848 nipype.utils INFO:
	 Running nipype version 1.2.3 (latest: 1.2.3)


INFO:nipype.utils:Running nipype version 1.2.3 (latest: 1.2.3)


Get the paths for the unthresholded image for each analysis team, for the given hypothesis.

In [2]:
input_dir = '../data-narps/proc/'
img_paths = fun.get_data_paths(input_dir)

For now, select the first N studies. 

In [3]:
# N_img = 3
# img_paths = img_paths[:N_img]

Convert the images to the dataset directory that is readable by NiMARE, the meta-analysis package. 

In [4]:
ds_dict = fun.extract_from_paths(img_paths, data=['path', 'coord'], sample_size=sample_size, 
                    level=level, height_control=height_control, cluster_threshold=cluster_threshold)

/opt/miniconda-latest/envs/neuro_py36/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
/opt/miniconda-latest/envs/neuro_py36/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/opt/miniconda-latest/envs/neuro_py36/lib/python3.6/site-packages/numpy/matrixlib/defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


## Coordinate-based meta-analysis
Using Activation Likelihood Estimation

In [5]:
# Unthresholded ALE image
img_ale, img_p, img_z = fun.run_ALE(ds_dict)

INFO:nimare.dataset:Shared path detected: "/home/neuro/test/data-narps/proc/hypo1_unthresh_"


In [6]:
# Thresholded ALE image
img_ale_thr = thresholding.map_threshold(img_ale, level=level, 
                                           height_control=height_control,
                                           cluster_threshold=cluster_threshold)[0]

## Image-based meta-analysis
Using a mixed-effects linear model

In [7]:
# Unthresholded MFX image
ma = nimare.meta.ibma.MFX_GLM()
res = ma.fit(ds=Dataset(ds_dict))
img_t_MFZ = res.get_map('t')

NameError: name 'nimare' is not defined

In [ ]:
# thresholded MFX image
ma = nimare.meta.ibma.MFX_GLM(cdt=.05, q=.05)
res = ma.fit(ds=Dataset(ds_dict))
img_t_MFX_thr = res.get_map('t')

In [ ]:
# # Thresholded MFX image
# img_t_MFX_thr = thresholding.map_threshold(img_t_MFX, level=level,
#                                            height_control=height_control,
#                                            cluster_threshold=cluster_threshold)[0]

In [ ]:
meta_analysis = { 
    #'ALE': img_ale,
    #'ALE thresholded': img_ale_thr,
    't MFX unthresholded': img_t_MFX,
    't MFX thresholded': img_t_MFX_thr,
}
for name, img in meta_analysis.items():
    plotting.plot_stat_map(img, title=name, cut_coords=cut_coords,
                          figure=plt.figure(figsize=(10,5)))   
    plt.savefig('../figures/results_%s.png' %name.replace(' ', '-'))